In [57]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch


In [58]:

# ============================================
# 1. LOAD AND PREPARE DATA
# ============================================
df = pd.read_json('/kaggle/input/dataset/data.jsonl', lines=True)
print(f"Original dataset: {len(df)} samples")
print(f"AI: {df['label'].sum()}, Human: {(df['label'] == 0).sum()}")

# Sample balanced subset (200 samples total - adjust as needed)
n_samples = 100  # per class

ai_samples = df[df['label'] == 1].sample(n=min(n_samples, len(df[df['label'] == 1])), random_state=42)
human_samples = df[df['label'] == 0].sample(n=min(n_samples, len(df[df['label'] == 0])), random_state=42)

# Combine and shuffle
small_df = pd.concat([ai_samples, human_samples]).sample(frac=1, random_state=42).reset_index(drop=True)
print(f"\nTraining with: {len(small_df)} samples")
print(f"AI: {small_df['label'].sum()}, Human: {(small_df['label'] == 0).sum()}")


Original dataset: 5410 samples
AI: 2742, Human: 2668

Training with: 200 samples
AI: 100, Human: 100


In [59]:

# ============================================
# 2. TRAIN-TEST SPLIT
# ============================================
train_texts, val_texts, train_labels, val_labels = train_test_split(
    small_df['content'].tolist(), 
    small_df['label'].tolist(),
    test_size=0.2, 
    random_state=42,
    stratify=small_df['label']
)

print(f"\nTrain samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")



Train samples: 160
Validation samples: 40


In [60]:

# ============================================
# 3. LOAD MODEL AND TOKENIZER
# ============================================
model_name = "openai-community/roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    ignore_mismatched_sizes=True
)


Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:

# ============================================
# 4. TOKENIZE DATA
# ============================================
def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        padding=False,  # Will pad dynamically during training
        truncation=True, 
        max_length=512
    )

# Create HuggingFace datasets
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [62]:

# ============================================
# 5. METRICS
# ============================================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [63]:

# ============================================
# 6. CUSTOM PROGRESS CALLBACK
# ============================================
from transformers import TrainerCallback
import sys
import time

class ProgressBarCallback(TrainerCallback):
    def __init__(self, num_epochs, num_steps_per_epoch):
        self.num_epochs = num_epochs
        self.num_steps_per_epoch = num_steps_per_epoch
        self.total_steps = num_epochs * num_steps_per_epoch
        self.current_step = 0
        self.current_epoch = 0
        self.epoch_start_time = None
        self.training_start_time = None
        self.best_accuracy = 0.0
        
    def on_train_begin(self, args, state, control, **kwargs):
        self.training_start_time = time.time()
        print("\n" + "="*70)
        print("🚀 TRAINING STARTED")
        print("="*70)
        print(f"Total Epochs: {self.num_epochs} | Steps per Epoch: {self.num_steps_per_epoch}")
        print(f"Total Steps: {self.total_steps}")
        print("="*70 + "\n")
        
    def on_epoch_begin(self, args, state, control, **kwargs):
        self.current_epoch = int(state.epoch) if state.epoch else 0
        self.epoch_start_time = time.time()
        print(f"\n{'='*70}")
        print(f"📚 EPOCH {self.current_epoch + 1}/{self.num_epochs}")
        print(f"{'='*70}")
        
    def on_step_end(self, args, state, control, **kwargs):
        self.current_step = state.global_step
        
        # Calculate progress
        epoch_progress = (state.global_step % self.num_steps_per_epoch) / self.num_steps_per_epoch
        total_progress = state.global_step / self.total_steps
        
        # Create progress bar
        bar_length = 40
        filled_length = int(bar_length * epoch_progress)
        bar = '█' * filled_length + '░' * (bar_length - filled_length)
        
        # Get current loss
        current_loss = state.log_history[-1].get('loss', 0) if state.log_history else 0
        
        # Calculate ETA
        elapsed = time.time() - self.epoch_start_time
        steps_done = (state.global_step % self.num_steps_per_epoch) + 1
        steps_remaining = self.num_steps_per_epoch - steps_done
        eta = (elapsed / steps_done) * steps_remaining if steps_done > 0 else 0
        
        # Print progress
        sys.stdout.write(f'\r')
        sys.stdout.write(f'[{bar}] {epoch_progress*100:.1f}% | Step {steps_done}/{self.num_steps_per_epoch} | Loss: {current_loss:.4f} | ETA: {eta:.0f}s')
        sys.stdout.flush()
        
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            print("\n")
            print(f"\n{'─'*70}")
            print("📊 VALIDATION RESULTS:")
            print(f"{'─'*70}")
            
            # Extract and display metrics
            eval_loss = metrics.get('eval_loss', 0)
            eval_acc = metrics.get('eval_accuracy', 0)
            eval_f1 = metrics.get('eval_f1', 0)
            eval_precision = metrics.get('eval_precision', 0)
            eval_recall = metrics.get('eval_recall', 0)
            
            # Update best accuracy
            if eval_acc > self.best_accuracy:
                self.best_accuracy = eval_acc
                best_marker = " 🌟 NEW BEST!"
            else:
                best_marker = ""
            
            print(f"  Accuracy:  {eval_acc*100:.2f}%{best_marker}")
            print(f"  Loss:      {eval_loss:.4f}")
            print(f"  F1 Score:  {eval_f1:.4f}")
            print(f"  Precision: {eval_precision:.4f}")
            print(f"  Recall:    {eval_recall:.4f}")
            print(f"{'─'*70}")
            
            # Progress bar for overall training
            overall_progress = (self.current_epoch + 1) / self.num_epochs
            bar_length = 50
            filled = int(bar_length * overall_progress)
            bar = '█' * filled + '░' * (bar_length - filled)
            print(f"\n📈 Overall Progress: [{bar}] {overall_progress*100:.1f}%")
            print(f"   Epochs Completed: {self.current_epoch + 1}/{self.num_epochs}")
            
    def on_train_end(self, args, state, control, **kwargs):
        total_time = time.time() - self.training_start_time
        minutes = int(total_time // 60)
        seconds = int(total_time % 60)
        
        print(f"\n\n{'='*70}")
        print("✅ TRAINING COMPLETE!")
        print(f"{'='*70}")
        print(f"  Total Time: {minutes}m {seconds}s")
        print(f"  Best Validation Accuracy: {self.best_accuracy*100:.2f}%")
        print(f"  Total Steps: {self.total_steps}")
        print(f"{'='*70}\n")


In [64]:

# ============================================
# 7. TRAINING CONFIGURATION
# ============================================
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Start with 3 epochs for small dataset
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=5,  # Log every 5 steps for more frequent updates
    eval_strategy="steps",  # Changed from "epoch" to "steps"
    eval_steps=10,  # Validate every 10 steps instead of every epoch
    save_strategy="steps",  # Save every 10 steps too
    save_steps=10,
    save_total_limit=3,  # Keep only 3 best checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    learning_rate=2e-5,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
    report_to="none",  # Disable wandb/tensorboard
    disable_tqdm=True  # Disable default progress bar
)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Calculate steps per epoch
steps_per_epoch = len(train_dataset) // training_args.per_device_train_batch_size
if len(train_dataset) % training_args.per_device_train_batch_size != 0:
    steps_per_epoch += 1

print(f"⚙️  Training Configuration:")
print(f"   • Validation every {training_args.eval_steps} steps")
print(f"   • Steps per epoch: {steps_per_epoch}")
print(f"   • Total validations per epoch: ~{steps_per_epoch // training_args.eval_steps}")

# Initialize custom progress callback
progress_callback = ProgressBarCallback(
    num_epochs=training_args.num_train_epochs,
    num_steps_per_epoch=steps_per_epoch
)

# ============================================
# 8. TRAIN MODEL
# ============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[progress_callback]  # Add custom callback
)

⚙️  Training Configuration:
   • Validation every 10 steps
   • Steps per epoch: 20
   • Total validations per epoch: ~2


/tmp/ipykernel_36/1514412300.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [65]:
trainer.train()



🚀 TRAINING STARTED
Total Epochs: 3 | Steps per Epoch: 20
Total Steps: 60


📚 EPOCH 1/3
[██████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 25.0% | Step 6/20 | Loss: 0.0000 | ETA: 155s{'loss': 2.4203, 'grad_norm': 139.97254943847656, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.25}
[████████████████████░░░░░░░░░░░░░░░░░░░░] 50.0% | Step 11/20 | Loss: 2.4203 | ETA: 115s{'loss': 2.7389, 'grad_norm': 91.70169067382812, 'learning_rate': 3.6000000000000003e-06, 'epoch': 0.5}
{'eval_loss': 3.4577457904815674, 'eval_accuracy': 0.45, 'eval_f1': 0.5217391304347826, 'eval_precision': 0.46153846153846156, 'eval_recall': 0.6, 'eval_runtime': 15.779, 'eval_samples_per_second': 2.535, 'eval_steps_per_second': 0.317, 'epoch': 0.5}



──────────────────────────────────────────────────────────────────────
📊 VALIDATION RESULTS:
──────────────────────────────────────────────────────────────────────
  Accuracy:  45.00% 🌟 NEW BEST!
  Loss:      3.4577
  F1 Score:  0.5217
  Precision: 0.4615
  Recall:    0.60

TrainOutput(global_step=60, training_loss=0.9697819451491038, metrics={'train_runtime': 898.0815, 'train_samples_per_second': 0.534, 'train_steps_per_second': 0.067, 'train_loss': 0.9697819451491038, 'epoch': 3.0})

In [66]:

# ============================================
# 8. EVALUATE
# ============================================
print("\n" + "="*60)
print("EVALUATION RESULTS")
print("="*60)

eval_results = trainer.evaluate()
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")



EVALUATION RESULTS
{'eval_loss': 1.1757831573486328, 'eval_accuracy': 0.575, 'eval_f1': 0.6530612244897959, 'eval_precision': 0.5517241379310345, 'eval_recall': 0.8, 'eval_runtime': 16.7231, 'eval_samples_per_second': 2.392, 'eval_steps_per_second': 0.299, 'epoch': 3.0}



──────────────────────────────────────────────────────────────────────
📊 VALIDATION RESULTS:
──────────────────────────────────────────────────────────────────────
  Accuracy:  57.50%
  Loss:      1.1758
  F1 Score:  0.6531
  Precision: 0.5517
  Recall:    0.8000
──────────────────────────────────────────────────────────────────────

📈 Overall Progress: [██████████████████████████████████████████████████] 100.0%
   Epochs Completed: 3/3
eval_loss: 1.1758
eval_accuracy: 0.5750
eval_f1: 0.6531
eval_precision: 0.5517
eval_recall: 0.8000
eval_runtime: 16.7231
eval_samples_per_second: 2.3920
eval_steps_per_second: 0.2990
epoch: 3.0000


In [67]:

# ============================================
# 9. SAVE MODEL
# ============================================
model.save_pretrained('./finetuned_roberta_detector')
tokenizer.save_pretrained('./finetuned_roberta_detector')
print("\n✅ Model saved to './finetuned_roberta_detector'")



✅ Model saved to './finetuned_roberta_detector'


In [69]:

# ============================================
# 10. TEST ON NEW SAMPLES
# ============================================
print("\n" + "="*60)
print("TESTING ON SAMPLE TEXTS")
print("="*60)

from transformers import pipeline

# Load your fine-tuned model
pipe = pipeline("text-classification", model='./finetuned_roberta_detector', device=0 if torch.cuda.is_available() else -1)

# Test samples
test_samples = [
    # Social Media / Casual Human Text
    ("Check out this amazing sunset I captured! #nature #photography", "Human"),
    ("Just finished my morning coffee ☕ Ready to tackle the day!", "Human"),
    ("Can't believe I forgot my keys again 🤦‍♂️ third time this week lol", "Human"),
    ("Anyone else watching the game tonight? LFG!! 🏈", "Human"),
    ("Tried that new restaurant downtown - totally worth the hype! The pasta was *chef's kiss*", "Human"),
    
    # Academic / Formal AI Text
    ("Statistical inference is the process of using data analysis to infer properties of an underlying distribution.", "AI"),
    ("Machine learning algorithms utilize patterns in data to make predictions without being explicitly programmed.", "AI"),
    ("The implementation of neural networks involves multiple layers of interconnected nodes that process information.", "AI"),
    ("Quantitative analysis demonstrates a significant correlation between the variables under investigation.", "AI"),
    ("The methodology employed in this study encompasses both qualitative and quantitative research approaches.", "AI"),
    
    # Conversational Human Text
    ("Honestly, I have no idea what I'm doing half the time but somehow it works out", "Human"),
    ("My dog just stole my sandwich right off the table 😂 can't even be mad", "Human"),
    ("Does anyone know a good place to buy cheap textbooks? College is expensive af", "Human"),
    
    # Technical AI Text
    ("The algorithm processes input data through convolutional layers to extract relevant features for classification.", "AI"),
    ("Database normalization reduces redundancy and improves data integrity through systematic decomposition of tables.", "AI"),
    ("Cloud computing infrastructure provides scalable resources through virtualization and distributed systems.", "AI"),
    
    # Creative Human Writing
    ("The rain tapped gently on my window as I curled up with my favorite book and hot chocolate", "Human"),
    ("Running late as usual, grabbed my bag and practically flew out the door", "Human"),
    ("Sometimes I wonder if my plants judge me for forgetting to water them 🌱", "Human"),
    
    # Formal AI Writing
    ("Contemporary research indicates that climate change significantly impacts global biodiversity patterns.", "AI"),
    ("Economic indicators suggest substantial fluctuations in market volatility during the observed period.", "AI"),
    ("The experimental results demonstrate statistically significant differences between the control and treatment groups.", "AI"),
]

for text, expected in test_samples:
    result = pipe(text)
    predicted = "AI" if result[0]['label'] == 'LABEL_1' else "Human"
    confidence = result[0]['score']
    
    status = "✅" if predicted == expected else "❌"
    print(f"\n{status} Expected: {expected} | Predicted: {predicted} ({confidence:.1%})")
    print(f"Text: {text[:100]}...")

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)


TESTING ON SAMPLE TEXTS


Device set to use cpu



✅ Expected: Human | Predicted: Human (51.5%)
Text: Check out this amazing sunset I captured! #nature #photography...

✅ Expected: Human | Predicted: Human (77.5%)
Text: Just finished my morning coffee ☕ Ready to tackle the day!...

✅ Expected: Human | Predicted: Human (81.6%)
Text: Can't believe I forgot my keys again 🤦‍♂️ third time this week lol...

✅ Expected: Human | Predicted: Human (91.7%)
Text: Anyone else watching the game tonight? LFG!! 🏈...

✅ Expected: Human | Predicted: Human (80.4%)
Text: Tried that new restaurant downtown - totally worth the hype! The pasta was *chef's kiss*...

❌ Expected: AI | Predicted: Human (77.9%)
Text: Statistical inference is the process of using data analysis to infer properties of an underlying dis...

❌ Expected: AI | Predicted: Human (54.4%)
Text: Machine learning algorithms utilize patterns in data to make predictions without being explicitly pr...

❌ Expected: AI | Predicted: Human (89.3%)
Text: The implementation of neural networks involve